In [1]:
from from_root import from_root
ROOT = from_root()

from dataclasses import dataclass
from datetime import datetime
import yaml
from typing import Optional
import pandas as pd


In [38]:
# Extract open order from PAPER
from ib_async import order, util
with get_ib('SNP', LIVE=False) as ib:
    oo = ib.reqAllOpenOrders()

In [ ]:
util.df(oo)

In [48]:
# Function to rename fields and convert the object to a DataFrame
def rename_fields(obj, field_map):
    def rename(obj):
        if isinstance(obj, list):
            # If the object is a list of objects, apply recursively to each item
            for item in obj:
                rename(item)
        else:
            # If the object is not a list, apply the renaming
            for old_field, new_field in field_map.items():
                if hasattr(obj, old_field):
                    value = getattr(obj, old_field)
                    setattr(obj, new_field, value)
                    delattr(obj, old_field)
    
    # Rename fields
    rename(obj)



In [68]:
import numpy as np

def expand_dataframe(df):
    # Create an empty DataFrame to store the expanded columns
    expanded_df = pd.DataFrame()
    
    # Convert the input to DataFrame if it's not already
    if not isinstance(df, pd.DataFrame):
        df = util.df(df)
    
    # Process each column in the DataFrame
    for col in df.columns:
        # Skip columns with all null values
        if df[col].isnull().all():
            continue
            
        # Get the first non-null value to determine the type
        sample_value = df[col].iloc[0]
        
        if isinstance(sample_value, list):
            # Handle list columns (like 'log' and 'fills')
            if len(sample_value) > 0:
                # For non-empty lists (like 'log' with TradeLogEntry)
                list_df = util.df(sample_value)
                if not list_df.empty:
                    expanded_df = pd.concat([expanded_df, list_df], axis=1)
        else:
            # Handle non-list objects (like Option, Order, OrderStatus)
            obj_df = util.df(df[col].tolist())
            expanded_df = pd.concat([expanded_df, obj_df], axis=1)
    
    # Remove duplicate columns
    expanded_df = expanded_df.loc[:, ~expanded_df.columns.duplicated()]
    
    # Remove columns with all null values or empty strings
    expanded_df = expanded_df.dropna(axis=1, how='all')  # Remove columns with all NaN
    
    # Remove columns with all empty strings
    empty_cols = expanded_df.columns[expanded_df.astype(str).eq('').all()]
    expanded_df = expanded_df.drop(columns=empty_cols)
    
    # Remove columns with string representation of empty lists
    empty_list_cols = expanded_df.columns[expanded_df.astype(str).eq('[]').all()]
    expanded_df = expanded_df.drop(columns=empty_list_cols)
    
    # Remove columns with all UNSET_DOUBLE values
    unset_cols = expanded_df.columns[expanded_df.apply(lambda x: (x == util.UNSET_DOUBLE).all())]
    expanded_df = expanded_df.drop(columns=unset_cols)
    
    return expanded_df

In [69]:
from ibfuncs import get_ib

with get_ib('SNP', LIVE=False) as ib:
    open_orders = ib.reqAllOpenOrders()

# Define the field name mapping
field_map = {
    "lastTradeDateOrContractMonth": "expiry",
    # Add more field mappings as needed
}

# Rename the fields in the object
df = expand_dataframe(open_orders)

In [ ]:
df